# FRI: Fast Reed Solomon IOP of Proximity

In [4]:
from sage.rings.finite_rings.all import *
from sage.rings.polynomial.all import *

def chunk_poly(poly, split_degree):
    chunks = poly.degree()//split_degree
    splits = [ poly.parent()(0) for _ in range(split_degree) ]
    indeterminate = poly.parent().gen()

    for (i,coef) in enumerate(poly.list()):
        ndx = i % split_degree
        deg = i // split_degree
        splits[ndx] = splits[ndx] + coef*(indeterminate**deg)

    pp = poly.parent()(0)

    for (j,mini_poly) in enumerate(splits):
        pp = pp + mini_poly.subs({ indeterminate : indeterminate** split_degree})*(indeterminate**j)
    assert pp == poly, "The reconstruction works"

    return splits

def fold(commit_poly, split_size, challenge, eval_domain):
    splits = chunk_poly(commit_poly, split_size)
    updated_domain = list(set([x**split_size for x in eval_domain]))
    new_poly = commit_poly.parent()(0)
    for i in range(split_size):
        new_poly = new_poly + (challenge**i)*splits[i]

    return (new_poly, updated_domain)
    

mult_order = 15*(2**27)
Fq = GF(mult_order+1)
Rx = PolynomialRing(Fq, "X")
eval_domain_size = 2**8
rate_factor = 4
k_factor = 4
max_degree = eval_domain_size // rate_factor

G = Fq.multiplicative_generator()
g = G**(mult_order // eval_domain_size)

eval_domain = [g^i for i in range(1, eval_domain_size+1)]

poly_commit = Rx.random_element(max_degree - 1);
print(poly_commit)
p,eval_domain_2 = fold(poly_commit, k_factor, 34, eval_domain)
print(f"{p}")
print(f"Updated eval domain length: {len(eval_domain_2)} vs. {len(eval_domain)}")

209670444*X^63 + 619210726*X^62 + 716484126*X^61 + 1026693660*X^60 + 1712135823*X^59 + 851189818*X^58 + 1790449754*X^57 + 1441568294*X^56 + 626094291*X^55 + 1513023179*X^54 + 2006254524*X^53 + 1181563468*X^52 + 1715892937*X^51 + 51554832*X^50 + 1138839230*X^49 + 1526384152*X^48 + 877485725*X^47 + 252070210*X^46 + 1583044088*X^45 + 405705708*X^44 + 28523152*X^43 + 642738217*X^42 + 1473155491*X^41 + 1105097866*X^40 + 999964422*X^39 + 262347000*X^38 + 410815183*X^37 + 1843152716*X^36 + 13796071*X^35 + 1864137813*X^34 + 1541289774*X^33 + 856000143*X^32 + 1533849335*X^31 + 1842944996*X^30 + 1024472393*X^29 + 344779029*X^28 + 406578779*X^27 + 1585503615*X^26 + 675840124*X^25 + 1494547900*X^24 + 560565813*X^23 + 1437514990*X^22 + 1521253208*X^21 + 667263069*X^20 + 90955195*X^19 + 1216034355*X^18 + 370720170*X^17 + 1616501690*X^16 + 334190186*X^15 + 1106470632*X^14 + 1745438653*X^13 + 146986405*X^12 + 1988533412*X^11 + 525012578*X^10 + 1839727791*X^9 + 1720393901*X^8 + 412361066*X^7 + 18865919